In [1]:
# Numpy is a library for working with Arrays
import numpy as np
print ("Your numpy version: %6.6s (need at least 1.7.1)" % np.__version__)

# SciPy implements many different numerical algorithms
import scipy as sp
print ("Your SciPy version: %6.6s (need at least 0.12.0)" % sp.__version__)

# Pandas makes working with data tables easier
import pandas as pd
print ("Your Pandas version: %6.6s (need at least 0.11.0)" % pd.__version__)

# Module for plotting
import matplotlib
print ("Your Mapltolib version: %6.6s (need at least 1.2.1)" % matplotlib.__version__)

# SciKit Learn implements several Machine Learning algorithms
import sklearn
print ("Your Scikit-Learn version: %6.6s (need at least 0.13.1)" % sklearn.__version__)

Your numpy version: 1.22.2 (need at least 1.7.1)
Your SciPy version:  1.7.1 (need at least 0.12.0)
Your Pandas version:  1.3.4 (need at least 0.11.0)
Your Mapltolib version:  3.4.3 (need at least 1.2.1)
Your Scikit-Learn version:  1.0.2 (need at least 0.13.1)


In [2]:
data = pd.read_csv('C:/Users/Kruti/Pitchbook/data/sample_data.csv')

In [3]:
data.shape

(10000, 9)

In [4]:
data = data[data.full_desc != '[N/A]']

In [5]:
data.head(3)

,Unnamed: 0,entity_name,full_desc,generated_keywords,verticals,industry_code,industry_sector,industry_group,keywords
0,0,Adtelligence,Provider of an e-commerce optimization and cus...,"['cyber intelligence', 'user retention', 'audi...","['TMT', 'AdTech', 'Big Data', 'Artificial Inte...","['Business/Productivity Software', 'Media and ...","['Information Technology', 'Business Products ...","['Software', 'Commercial Services']","['data parameters', 'offer optimization', 'off..."
1,1,Mojo (Educational Software),Developer of sports application designed to ma...,"['rafting', 'coaching software', 'sports train...","['Mobile', 'EdTech', 'SaaS']","['Educational and Training Services (B2C)', 'E...","['Consumer Products and Services (B2C)', 'Info...","['Services (Non-Financial)', 'Software', 'Soft...","['sports and recreation', 'youth athletes', 'p..."
2,2,Status Money,Developer of a banking application designed to...,"['digital banking platform', 'financial health...","['Big Data', 'FinTech', 'Artificial Intelligen...","['Application Software', 'Other Financial Serv...","['Information Technology', 'Financial Services...","['Software', 'Other Financial Services', 'Soft...","['cashbacks', 'bitcoin rewards', 'financial se..."


In [25]:
data.shape

(9905, 9)

In [7]:
df_name = data.filter(['entity_name', 'generated_keywords'], axis = 1)

In [8]:
df_desc = data.filter(['full_desc'], axis = 1)

In [9]:
df_desc = df_desc.astype(str)

In [10]:
#print(df_desc[97])

In [11]:
#df_data.head(3)

NameError: name 'df_data' is not defined

In [12]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction

In [13]:
stopwords = nltk.corpus.stopwords.words('english')
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [14]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [15]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in df_desc.index:
    #print(i)
    allwords_stemmed = tokenize_and_stem(df_desc['full_desc'][i]) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(df_desc['full_desc'][i])
    totalvocab_tokenized.extend(allwords_tokenized)

In [16]:
len(totalvocab_stemmed)

414087

In [17]:
len(totalvocab_tokenized)

414087

In [20]:
totalvocab_stemmed[200:205]

['dine', 'experi', 'the', 'compani', 'offer']

In [21]:
totalvocab_tokenized[200:205]

['dining', 'experience', 'the', 'company', 'offers']

In [22]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 414087 items in vocab_frame


In [ ]:
vocab_frame.head(5)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.99, max_features=200000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df_desc['full_desc']) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

C:\Users\Kruti\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(


Wall time: 16.4 s
(9905, 484)


In [26]:
terms = tfidf_vectorizer.get_feature_names_out()

In [27]:
terms

array(["'s", "'s applic", "'s platform", "'s platform offer",
       "'s platform provid", "'s product", "'s product includ",
       "'s servic", "'s servic includ", "'s softwar", "'s technolog",
       'access', 'accessori', 'account', 'achiev', 'acquisit', 'activ',
       'addit', 'administr', 'advanc', 'advertis', 'advisori', 'afford',
       'agenc', 'agricultur', 'air', 'allow', 'allow user', 'altern',
       'analysi', 'analyt', 'analyz', 'ani', 'applic', 'applic design',
       'area', 'artifici', 'artifici intellig', 'assess', 'asset',
       'assist', 'autom', 'automot', 'avail', 'bank', 'base', 'benefit',
       'better', 'beverag', 'book', 'brand', 'build', 'busi', 'buy',
       'california', 'canada', 'capit', 'car', 'card', 'care', 'cater',
       'center', 'chain', 'channel', 'chemic', 'china', 'clean', 'client',
       'clinic', 'cloud', 'cloud-bas', 'collabor', 'collect', 'combin',
       'commerci', 'communic', 'communiti', 'compani', "compani 's",
       "compani 's a

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [29]:
from sklearn.cluster import KMeans
num_clusters = 15
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

Wall time: 1min 12s


In [ ]:
import joblib


In [ ]:
#pip install sklearn.externals

In [30]:
import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

#joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [31]:
cluster = pd.DataFrame(clusters)

In [32]:
cluster.head(3)

,0
0,7
1,10
2,10


In [33]:
frame = pd.DataFrame(index = [clusters], columns = [df_name, cluster])

In [34]:
frames = pd.merge(df_name, cluster, left_index=True, right_index=True)

In [35]:
frames.rename(columns = {0: "Cluster"}, inplace = True)

In [36]:
frames.head()

,entity_name,generated_keywords,Cluster
0,Adtelligence,"['cyber intelligence', 'user retention', 'audi...",7
1,Mojo (Educational Software),"['rafting', 'coaching software', 'sports train...",10
2,Status Money,"['digital banking platform', 'financial health...",10
3,Coya Restaurant,"['beverage products', 'cloud kitchen platform'...",11
4,Trista,"['publishing services firm', 'publishing servi...",7


In [ ]:
frames.tail()

In [37]:
frames['Cluster'].value_counts() 

14    2009
7     1404
2     1042
0      825
8      790
11     646
9      529
5      500
13     479
4      448
12     368
10     295
3      189
6      187
1      100
Name: Cluster, dtype: int64

In [38]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.iloc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frames.ix[i]['entity_name'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words:

ValueError: invalid literal for int() with base 10: 'product'